# Préparation des données

## Chargement des données

In [3]:
import pandas as pd
file_path = '../data/relatives.xlsx'
sheet_name = 'data'
df = pd.read_excel(file_path, sheet_name=sheet_name)

In [ ]:
df.head()

In [ ]:
df.describe()

## Prétraitement des données :



### Normalisation:

cette partie permet de nettoyer et uniformiser des textes en remplaçant des caractères spéciaux par leurs équivalents standards.

In [4]:
import re

def replace_characters(match: re.Match) -> str:
    char = match.group(0)
    replacements = {'’': "'",'´': "'",'`': "'",'‘': "'",'«': '"','»': '"','“': '"','”': '"','–': '-','—': '-','…': ' ',u'\xa0': ' ',}
    return replacements[char]

def normalize_text(text: str) -> str:
    pattern = r'[’´`‘«»“”–—…]'
    return re.sub(pattern, replace_characters, text).strip()

In [ ]:
# test
text = "un text avec des caractères spéciaux ’´`‘«»“”–—…"
print(normalize_text(text))

Ainsi, nous allons remplacer les caractères spéciaux par leurs équivalents standards.

In [5]:
# apply to the dataframe
df = df.map(normalize_text)

### Tokenization:

Cette partie permet de découper le texte en mots

In [6]:
# chargement de la librairie nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/amina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**RegexpTokenizer** : permet de diviser une chaîne de caractères en sous-chaînes en utilisant une expression régulière.

In [7]:
from nltk import RegexpTokenizer

def tokenizer(text):
  return RegexpTokenizer(r'''\w'|\w+|[^\w\s]''').tokenize(text)

La fonction **tokenizer_list** permet de diviser une séquence de texte en sous-séquences en utilisant le caractère '|' comme délimiteur, puis de segmenter chaque sous-séquence en mots et ponctuations.

In [8]:
def tokenizer_list(text):
  texts = text.split("|")
  result = []
  for t in texts:
    result.append( tokenizer(t) )

  return result

In [ ]:
# Test
text = "Bonjour tout le monde!|il s'agit d'un test."
tokenizer_list(text)

Application de la tokenization sur les données 

In [9]:
# Application de la fonction tokenizer sur les colonnes Phrase et PSR
df['tokens'] = df['texts'].apply(tokenizer)
df['PSR'] = df['PSR'].apply(tokenizer_list)
df['annotation'] = df['annotation'].apply(lambda x : [t.strip() for t in x.split("|")])

In [ ]:
df.head()

#### vérification de la tokenization

In [10]:
# Fonction pour vérifier la taille des listes dans les colonnes 'PSR' et 'annotation'
def check_list_sizes(row):
    psr_len = len(row['PSR'])
    annotation_len = len(row['annotation'])
    return psr_len == annotation_len

In [11]:
# Appliquer la vérification à chaque ligne du DataFrame
df['size_match'] = df.apply(check_list_sizes, axis=1)

In [12]:
df[df['size_match'] == False]

,texts,PSR,annotation,tokens,size_match


In [ ]:
# Supprimer des lignes où la taille des listes dans les colonnes 'PSR' et 'annotation' ne correspond pas
df = df[df['size_match'] == True]

### Labilisation:

L'objectif de cette partie est de transformer les labels en valeurs numériques et aligner les labels avec le texte tokenisé.

In [13]:
# Fonction pour annoter le texte 
# permet de labiliser en mode séquentiel ou non
def annotate(text, psr_lst, tag_lst, mode_seq=False):
    # Initialiser la liste des annotations avec des 0
    annotation = [0] * len(text)
    
    
    # Définir les labels pour chaque tag
    labels = (
        {'O': 0, 'det': 1, 'appo': 3, 'ambiguë': 5} if mode_seq 
        else {'O': 0, 'det': 1, 'appo': 2, 'ambiguë': 3}
    )
    
    # Trouver la longueur du texte
    text_len = len(text)
    if len(psr_lst) != len(tag_lst):
        raise ValueError("Les listes psr_lst et tag_lst doivent avoir la même taille")
    
    # Parcourir chaque psr dans psr_lst
    for i, psr in enumerate(psr_lst):
        psr_len = len(psr)  # Longueur de la sous-séquence psr
        
        tag = tag_lst[i]  # Etiquette correspondante
        
        # Parcourir le texte pour trouver des correspondances avec psr
        for j in range(text_len - psr_len + 1):
            if text[j:j + psr_len] == psr:
                annotation[j] = labels[tag] # Labelliser le token correspondant
                if mode_seq: # Si le mode séquentiel est activé, labelliser les tokens suivants
                  for k in range(j + 1, j + psr_len):
                    annotation[k] = labels[tag] + 1
                break

    return annotation

In [27]:
words = tokenizer("Vous passerez, nous passerons sans doute des jours et des nuits sur des sujets qui touchent le quotidien de nos concitoyens et qui auront chacun leur importance.")
psr = tokenizer_list("qui touchent le quotidien de nos concitoyens|qui auront chacun leur importance")
tag = 'appo|appo'.split('|')

print(words, len(words))
print(psr)
print(tag)

['Vous', 'passerez', ',', 'nous', 'passerons', 'sans', 'doute', 'des', 'jours', 'et', 'des', 'nuits', 'sur', 'des', 'sujets', 'qui', 'touchent', 'le', 'quotidien', 'de', 'nos', 'concitoyens', 'et', 'qui', 'auront', 'chacun', 'leur', 'importance', '.'] 29
[['qui', 'touchent', 'le', 'quotidien', 'de', 'nos', 'concitoyens'], ['qui', 'auront', 'chacun', 'leur', 'importance']]
['appo', 'appo']


In [28]:
# test de la fonction annotate

labels = annotate(words, psr,tag )
labels_seq = annotate(words, psr,tag, mode_seq=True)
label_names = ['O', 'DET', 'APPO', 'AMBIGUE']
label_names_seq = ['O', 'B-DET', 'I-DET', 'B-APPO', 'I-APPO', 'B-AMBIGUE','I-AMBIGUE']


line1 = ""
line2 = ""
line3 = ""

for word, label, label_seq in zip(words, labels, labels_seq):
    full_label = label_names[label]
    full_label_seq = label_names_seq[label_seq]
    
    max_length = max(len(word), len(full_label), len(full_label_seq))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)
    line3 += full_label_seq + " " * (max_length - len(full_label_seq) + 1)

# Affichage aligné des tokens et de leurs tags
print(line1)
print(line2)
print(line3)

Vous passerez , nous passerons sans doute des jours et des nuits sur des sujets qui    touchent le     quotidien de     nos    concitoyens et qui    auront chacun leur   importance . 
O    O        O O    O         O    O     O   O     O  O   O     O   O   O      APPO   O        O      O         O      O      O           O  APPO   O      O      O      O          O 
O    O        O O    O         O    O     O   O     O  O   O     O   O   O      B-APPO I-APPO   I-APPO I-APPO    I-APPO I-APPO I-APPO      O  B-APPO I-APPO I-APPO I-APPO I-APPO     O 


In [14]:
# Appliquer la fonction annotate à chaque ligne du DataFrame pour créer la colonne psr_tags
df['psr_tags'] = df.apply(lambda row: annotate(row['tokens'], row['PSR'], row['annotation']), axis=1)

In [15]:
# Appliquer la fonction annotate à chaque ligne du DataFrame pour créer la colonne psr_tags
df['psr_seq_tags'] = df.apply(lambda row: annotate(row['tokens'], row['PSR'], row['annotation'], mode_seq=True), axis=1)

In [16]:
# Supprimer les colonnes PSR et annotation
df.drop(columns=['texts','PSR','annotation','size_match'], inplace=True)



# reinstialiser l'index
df.reset_index(names='id' ,inplace=True)


In [32]:

df.head()

,id,tokens,psr_tags,psr_seq_tags
0,0,"[Survivante, de, la, Shoah, ,, elle, a, su, pu...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, ..."
1,1,"[la, Constitution, permet, au, Président, de, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,"[Les, heures, que, nous, vivons, sont, de, cel...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0]"
3,3,"[Je, veux, aujourd, ', hui, vous, parler, du, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, ..."
4,4,"[Pour, d', autres, ,, au, nombre, desquels, je...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, ..."


## Création Dataset Hugging Face

### Découpage du jeu de données en train, validation et test

In [33]:
#split the dataset into train , validation and test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [34]:
# Supprimer les colonnes d'index auto-générées si elles existent
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [35]:
train_df.head()

,id,tokens,psr_tags,psr_seq_tags
0,16,"[je, demanderai, au, Premier, ministre, d', as...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, ..."
1,263,"[Nos, institutions, sont, en, effet, solides, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,217,"[Nous, devons, faire, en, sorte, que, nos, dis...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,435,"[Le, groupe, UDI, ,, Agir, et, indépendants, p...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,63,"[Sieyès, et, Mirabeau, ne, désertèrent, pas, s...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


### Préparation des datasets

In [21]:
from datasets import Dataset, Features, Sequence, Value, ClassLabel

In [36]:
# Définir les Features pour chaque colonne du DataFrame
features = Features({
    'id': Value(dtype='int32', id=None),
    'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
    'psr_tags': Sequence(feature=ClassLabel(names=['O', 'DET', 'APPO', 'AMBIGUE'], id=None), length=-1, id=None),
    'psr_seq_tags': Sequence(feature=ClassLabel(names=['O', 'B-DET', 'I-DET', 'B-APPO', 'I-APPO', 'B-AMBIGUE','I-AMBIGUE'], id=None), length=-1, id=None)
})

In [37]:
# Convertir le DataFrame en Dataset Hugging Face avec les Features spécifiés
train_ds = Dataset.from_pandas(train_df, features=features)
val_ds = Dataset.from_pandas(val_df, features=features)
test_ds = Dataset.from_pandas(test_df, features=features)

In [38]:
train_ds

Dataset({
    features: ['id', 'tokens', 'psr_tags', 'psr_seq_tags'],
    num_rows: 392
})

In [39]:
from datasets import DatasetDict

# Recréation du DatasetDict
dataset = DatasetDict({
    'train': train_ds,
    'validation': val_ds,
    'test': test_ds
})

In [40]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'psr_tags', 'psr_seq_tags'],
        num_rows: 392
    })
    validation: Dataset({
        features: ['id', 'tokens', 'psr_tags', 'psr_seq_tags'],
        num_rows: 99
    })
    test: Dataset({
        features: ['id', 'tokens', 'psr_tags', 'psr_seq_tags'],
        num_rows: 55
    })
})

### Sauvegarde des datasets et publication sur Hugging Face

In [41]:
dataset_name = 'relatives_psr'

In [30]:
from huggingface_hub import notebook_login

notebook_login()

In [42]:
# Enregistrer le DatasetDict dans le hub Hugging Face
dataset.push_to_hub(dataset_name)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/5.32k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/djamina/relatives_psr/commit/1f7c7701776a5850f65f8e7ff685d1824d6a3188', commit_message='Upload dataset', commit_description='', oid='1f7c7701776a5850f65f8e7ff685d1824d6a3188', pr_url=None, pr_revision=None, pr_num=None)